# Conduct snowline accuracy assessment

Rainey Aberle

October 2022

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
import rasterio as rio
import xarray as xr
import ee
import wxee as wx
import glob
import geopandas as gpd
import sys
from scipy import stats
import skimage.io
from skimage import feature
from shapely.geometry import Point, LineString, shape, MultiPolygon, Polygon
from shapely.ops import split, unary_union, polygonize, nearest_points
import pandas as pd

In [ ]:
# path to snow-cover-mapping
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'
# names of study sites
site_names = ['Gulkana'] #['Gulkana', 'SCascade', 'Sperry', 'Wolverine']
# path for output figures
figures_out_path = base_path+'figures/'

# add path to functions
sys.path.insert(1, base_path+'functions/')
import ps_pipeline_utils as f

In [ ]:
# Authenticate Google Earth Engine (GEE)
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# -----Loop through sites
results_df = pd.DataFrame(columns=['study_site', 'datetime', 'snowline_obs', 'snowline_est', 'distances'])
for site_name in site_names:    

    # define path to classified snow images
    im_path = base_path + '../study-sites/' + site_name + '/imagery/PlanetScope/'

    # define path to digitized snow lines
    sl_obs_path = base_path + '../snowline-package/' + site_name + '/snowlines/'

    # load AOI as gpd.GeoDataFrame
    AOI_fn = base_path + '../../GIS_data/RGI_outlines/' + site_name + '_RGI.shp'
    AOI = gpd.read_file(AOI_fn)

    # query GEE for DEM
    DEM, AOI_UTM = f.query_GEE_for_DEM(AOI)

    # load observed snow line shapefile names
    sl_obs_fns = glob.glob(sl_obs_path + '*.shp')
    sl_obs_fns.sort() # sort chronologically

    # load estimated snowlines
    sl_est_fn = im_path + 'snowlines/' + site_name + '_sl_elevs_old.pkl'
    sl_ests = pd.read_pickle(sl_est_fn)

    # initialize variables
    sl_obs_elevs = [None]*len(sl_obs_fns) # observed snow elevations
    datetimes = [None]*len(sl_obs_fns) # image datetimes

    # loop through observed snow lines
    for sl_obs_fn in sl_obs_fns:

        # -----Load datasets
        # load observed snow line
        sl_obs = gpd.read_file(sl_obs_fn)
        # extract date from filename
        date = sl_obs_fn.split('/'+site_name+'_')[1][0:11]
        datetime = np.datetime64(date[0:4]+ '-' + date[4:6] + '-' + date[6:8] + ' ' + date[9:11] + ':00:00')
        # load estimated snowline
        sl_est = sl_ests.loc[sl_ests.datetime==datetime]
        # reproject snow line to UTM
        sl_obs_UTM = sl_obs.to_crs(str(AOI_UTM.crs.to_epsg()))
        # open adjusted image of the same date
        im_adj_fn = glob.glob(im_path + 'adjusted-filtered/*' + date + '*.tif')[0] # define file name
        im_adj = rxr.open_rasterio(im_adj_fn) # open image as xarray.DataArray
        im_adj = im_adj.where(im_adj!=-9999) # remove no data values
        im_adj = im_adj / 1e4
         # open classified image from the same date
        im_classified_fn = glob.glob(im_path + 'classified/*' + date + '*.tif')[0] # define file name
        im_classified = rxr.open_rasterio(im_classified_fn) # open image as xarray.DataArray
        im_classified = im_classified.where(im_classified!=-9999)
        
        # -----Split line depending on distance between points
        max_dist = 100 # m
        line = sl_obs_UTM.geometry[0]
        first_point = Point(line.coords.xy[0][0], line.coords.xy[1][0])
        points = [Point(line.coords.xy[0][i], line.coords.xy[1][i]) for i in np.arange(0,len(line.coords.xy[0]))]
        isplit = [0] # point indices where to split the line
        for i, p in enumerate(points):
            if i!=0:
                dist = p.distance(points[i-1])
                if dist > max_dist:
                    isplit.append(i)
        isplit.append(len(points)) # add ending point to complete the last line
        line_split = [] # initialize split lines
        # loop through split indices
        if isplit:
            for i, p in enumerate(isplit[:-1]):
                if isplit[i+1]-isplit[i] > 1: # must have at least two points to make a line
                    line_split = line_split + [LineString(points[isplit[i]:isplit[i+1]])]
        else:
            line_split = line
    
        # -----Regrid the observed snowlines to equal spacing
        dx = 30 # point spacing
        points_regrid = []
        for line in line_split:
            distances = np.arange(0, line.length, dx)
            line_points = [line.interpolate(distance) for distance in distances] + [first_point]
            # filter points outside the AOI
            IAOI = np.where(np.array([p.within(AOI_UTM.geometry[0]) for p in line_points], dtype=int) ==1)[0]
            points_AOI = [line_points[i] for i in IAOI]
            points_regrid = points_regrid + [p for p in points_AOI]

        # -----Calculate distance between each observed snowline point and the closest estimated snowline point
        distances = np.zeros(len(points_regrid))
        for i, p in enumerate(points_regrid):
            # find nearest point
            nearest_point = nearest_points(sl_est.snowlines_coords[0][0], p)[0]
            # calculate distance between points
            distances[i] = p.distance(nearest_point)
            
        # -----Display results
        plt.figure(figsize=(8, 8))
        plt.imshow(np.dstack([im_adj.data[2], im_adj.data[1], im_adj.data[0]]), 
                   extent=(np.min(im_adj.x.data), np.max(im_adj.x.data), np.min(im_adj.y.data), np.max(im_adj.y.data)))
        plt.plot([p.coords.xy[0][0] for p in points_regrid], 
                 [p.coords.xy[1][0] for p in points_regrid], '.c', label='observed')
        plt.plot(*sl_est.snowlines_coords[0][0].coords.xy, '.m', label='estimated')
        plt.grid()
        plt.title(datetime)
        plt.show()
        
        # compile results in df
        result_df = pd.DataFrame({'study_site': site_name, 
                                  'datetime': datetime, 
                                  'snowline_obs': [points_regrid], 
                                  'snowline_est': [sl_est.snowlines_coords[0][0]], 
                                  'distances': [distances]})
        # concatenate to results_df
        results_df = pd.concat([results_df, result_df])


In [ ]:
fig, ax = plt.subplots()
ax.imshow(np.dstack([im_adj.data[2], im_adj.data[1], im_adj.data[0]]), 
                   extent=(np.min(im_adj.x.data), np.max(im_adj.x.data), np.min(im_adj.y.data), np.max(im_adj.y.data)))
AOI_UTM.plot(ax=ax, facecolor='none', edgecolor='m')
plt.show()